In [1]:
#IPython extension to reload modules before executing user code.
#'autoreload' reloads modules automatically before entering the execution of code typed at the IPython prompt.
%load_ext autoreload
%autoreload 2

In [2]:
import os
import gpustat

stats = gpustat.GPUStatCollection.new_query()
ids = map(lambda gpu: int(gpu.entry['index']), stats)
ratios = map(lambda gpu: float(gpu.entry['memory.used'])/float(gpu.entry['memory.total']), stats)
bestGPU = min(zip(ids, ratios), key=lambda x: x[1])[0]
bestGPU = 0 # manualy take the first GPU

print("setGPU: Setting GPU to: {}".format(bestGPU))
os.environ['CUDA_DEVICE_ORDER'] = 'PCI_BUS_ID'
os.environ['CUDA_VISIBLE_DEVICES'] = str(bestGPU)

setGPU: Setting GPU to: 0


In [3]:
!pip install -q keract
!pip install -q sklearn
import numpy
from keract import get_activations
import keras
from keras import backend as K
from keras import optimizers, regularizers
from keras.callbacks import LearningRateScheduler
from keras.datasets import cifar10 # we can use also cifar100
from keras.layers import Dense
from keras.models import Model
from keras.layers import Input, BatchNormalization, AveragePooling2D, ZeroPadding2D, LeakyReLU, GlobalAveragePooling2D, Dropout, Flatten, Activation
from keras.layers.convolutional import Conv2D, MaxPooling2D
from keras.models import Sequential, load_model
from keras.regularizers import l2
from keras.utils import plot_model
from keras.utils.vis_utils import model_to_dot
import matplotlib.pyplot as plt
import scipy
import sklearn
import sklearn.metrics
import tensorflow as tf
import time
from IPython.display import SVG

Using TensorFlow backend.


In [4]:
# LOAD DATA
num_classes = 10
# The data, split between train and test sets:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

# Convert class vectors to binary class matrices.
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255

x_train shape: (50000, 32, 32, 3)
50000 train samples
10000 test samples


### Loading models

In [5]:
# LOAD MODELS
#model A: sans regularization
modelA = load_model('../models/modelA.h5')
modelA.load_weights('../models/modelA_weights_epoch50.h5')
#model A: avec regularisation
modelB = load_model('../models/modelB.h5')
modelB.load_weights('../models/modelB_weights_epoch50.h5')

In [6]:
# EVALUATION
# Final evaluation of the models
scoresA = modelA.evaluate(x_test, y_test, verbose=1)
print('Test loss:', scoresA[0])
print('Test accuracy:', scoresA[1])
print("Model 1 CNN Error: %.2f%%" % (100-scoresA[1]*100))
scoresB = modelB.evaluate(x_test, y_test, verbose=1)
print('Test loss:', scoresB[0])
print('Test accuracy:', scoresB[1])
print("Model B CNN Error: %.2f%%" % (100-scoresB[1]*100))

UnknownError: Failed to get convolution algorithm. This is probably because cuDNN failed to initialize, so try looking to see if a warning log message was printed above.
	 [[{{node conv2d_1/convolution}} = Conv2D[T=DT_FLOAT, data_format="NCHW", dilations=[1, 1, 1, 1], padding="SAME", strides=[1, 1, 1, 1], use_cudnn_on_gpu=true, _device="/job:localhost/replica:0/task:0/device:GPU:0"](conv2d_1/convolution-0-TransposeNHWCToNCHW-LayoutOptimizer, conv2d_1/kernel/read)]]
	 [[{{node metrics/acc/Mean/_173}} = _Recv[client_terminated=false, recv_device="/job:localhost/replica:0/task:0/device:CPU:0", send_device="/job:localhost/replica:0/task:0/device:GPU:0", send_device_incarnation=1, tensor_name="edge_127_metrics/acc/Mean", tensor_type=DT_FLOAT, _device="/job:localhost/replica:0/task:0/device:CPU:0"]()]]

### Métrique Linéaire

Regarder la métrique spéciale de l'article pour comparer ce qu'apporte un neurone par rapport à tous les autres neurones de la couche (! restriction linéaire)
* https://arxiv.org/pdf/1706.05806.pdf

In [ ]:
nombreDImagesDActivation = 50
# Pour un echantillon test
Echantillon = []
choix = numpy.random.choice(x_test.shape[0], nombreDImagesDActivation)
Echantillon = x_test[choix, :, : , :]
# On regarde l'activation des neurones
IMA = get_activations(modelA, Echantillon)
# on affiche les noms des differentes couches
print(IMA.keys())

In [ ]:
def SVCCA(IM, layerName):
    SVCCA = 0
    
    #nombre d'images utilisées pour l'activation:
    numActivations = len(IM[layerName])
    #print('assert: '+str(numActivations==nombreDImagesDActivation))
    
    #nombre de neurones dans la couche
    numNeurons = numpy.size(IM[layerName][0])
    print('number of neurons for the layer: '+str(numNeurons))
    
    # on selectionne 2 groupes de neurones X et Y à comparer
    for activation in IM[layerName]:
        print(numpy.size(activation))
    # X = matrix [neurones x activation]
    
    
    #SVD
    u, s, vh = numpy.linalg.svd(X)
    X_ = s*vh
    u, s, vh = numpy.linalg.svd(Y)
    Y_ = s*vh
    #CCA
    cca = sklearn.cross_decomposition.CCA(n_components=2, scale=True, max_iter=500, tol=1e-06, copy=True)
    cca.fit(X_,Y_)
    X_c, Y_c = cca.transform(X, Y)
    corr = score(X_,Y_)
    
    SVCCA = corr
    
    #--------------------------------------------------------
    '''
    NeuronsSelected = numpy.zeros((NombrePairs,2))  # initialize
    for i in range(NombrePairs):
        NeuronsSelected[i] = numpy.ceil(numpy.random.rand(2)*numNeurons)  # on prend 2 neurones selectionnées aleatoirement
        # les 2 neurones doivent être différents
        while NeuronsSelected[i][0] == NeuronsSelected[i][1]:
            NeuronsSelected[i] = numpy.ceil(numpy.random.rand(2)*numNeurons)
    MutualInfo = numpy.zeros((NombrePairs,1))  # initialization des resultats
    for j in range(NombrePairs):
        X = numpy.zeros((numActivations,1))  # initialization du vecteur X
        Y = numpy.zeros((numActivations,1))  # initialization du vecteur y
        i = 0
        for activation in IM[layerName]:   # pour chaque element de l'echantillon (= chaque activation)
            activation = activation.flatten()
            X[i] = activation[int(NeuronsSelected[j][0])-1] # on regarde l'activation du neurone selectionné
            Y[i] = activation[int(NeuronsSelected[j][1])-1]
            if X[i] > 0:
                X[i] = 1
            else:
                X[i] = 0
            if Y[i] > 0:
                Y[i] = 1
            else:
                Y[i] = 0
            i = i+1
        X = X.flatten()
        Y = Y.flatten()
        MutualInfo[j] = sklearn.metrics.normalized_mutual_info_score(X,Y,average_method='max')'''
    return SVCCA

In [ ]:
List_of_Layers_A = ['conv2d_1/Relu:0', 'conv2d_2/Relu:0','max_pooling2d_1/MaxPool:0', 'conv2d_3/Relu:0', 'conv2d_4/Relu:0', 'max_pooling2d_2/MaxPool:0', 'flatten_1/Reshape:0', 'dense_1/Relu:0', 'dense_2/Softmax:0']
# model A a gauche
print('\n Model A :\n')
for layer in List_of_Layers_A:
    #nom de la couche
    print('couche :', str(layer))
    #MI
    Mesure_SVCCA = SVCCA(IMA, layer)
    print('Mesure_SVCCA : '+str(Mesure_SVCCA))